Описание задачи

Небольшой интернет-магазин попросил вас добавить ранжирование товаров в блок "Смотрели ранее" - в нем теперь надо показывать не последние просмотренные пользователем товары, а те товары из просмотренных, которые он наиболее вероятно купит. Качество вашего решения будет оцениваться по количеству покупок в сравнении с прошлым решением в ходе А/В теста, т.к. по доходу от продаж статзначимость будет достигаться дольше из-за разброса цен. Таким образом, ничего заранее не зная про корреляцию оффлайновых и онлайновых метрик качества, в начале проекта вы можете лишь постараться оптимизировать recall@k и precision@k.

Это задание посвящено построению простых бейзлайнов для этой задачи: ранжирование просмотренных товаров по частоте просмотров и по частоте покупок. Эти бейзлайны, с одной стороны, могут помочь вам грубо оценить возможный эффект от ранжирования товаров в блоке - например, чтобы вписать какие-то числа в коммерческое предложение заказчику, а с другой стороны, могут оказаться самым хорошим вариантом, если данных очень мало (недостаточно для обучения даже простых моделей).

Входные данные

Вам дается две выборки с пользовательскими сессиями - id-шниками просмотренных и id-шниками купленных товаров. Одна выборка будет использоваться для обучения (оценки популярностей товаров), а другая - для теста.

В файлах записаны сессии по одной в каждой строке. Формат сессии: id просмотренных товаров через , затем идёт ; после чего следуют id купленных товаров (если такие имеются), разделённые запятой. Например, 1,2,3,4; или 1,2,3,4;5,6.

Гарантируется, что среди id купленных товаров все различные.

Важно:

- Сессии, в которых пользователь ничего не купил, исключаем из оценки качества.
-    Если товар не встречался в обучающей выборке, его популярность равна 0.
-    Рекомендуем разные товары. И их число должно быть не больше, чем количество различных просмотренных пользователем товаров.
-    Рекомендаций всегда не больше, чем минимум из двух чисел: количество просмотренных пользователем товаров и k в recall@k / precision@k.

Задание

1.    На обучении постройте частоты появления id в просмотренных и в купленных (id может несколько раз появляться в просмотренных, все появления надо учитывать)
2. Реализуйте два алгоритма рекомендаций:

- сортировка просмотренных id по популярности (частота появления в просмотренных),
- сортировка просмотренных id по покупаемости (частота появления в покупках).

3. Для данных алгоритмов выпишите через пробел AverageRecall@1, AveragePrecision@1, AverageRecall@5, AveragePrecision@5 на обучающей и тестовых выборках, округляя до 2 знака после запятой. Это будут ваши ответы в этом задании. Посмотрите, как они соотносятся друг с другом. Где качество получилось выше? Значимо ли это различие? Обратите внимание на различие качества на обучающей и тестовой выборке в случае рекомендаций по частотам покупки.

Если частота одинаковая, то сортировать нужно по возрастанию момента просмотра (чем раньше появился в просмотренных, тем больше приоритет)

In [2]:
import pandas as pd
import numpy as np

In [3]:
train_data = pd.read_csv('train.txt', sep=';', header=None, names=['watched', 'bought'])
test_data = pd.read_csv('test.txt', sep=';', header=None, names=['watched', 'bought'])
train_data.head()

,watched,bought
0,"0,1,2,3,4,5",NaN
1,"9,10,11,9,11,12,9,11",NaN
2,"16,17,18,19,20,21",NaN
3,"24,25,26,27,24",NaN
4,"34,35,36,34,37,35,36,37,38,39,38,39",NaN


In [4]:
test_data.head()

,watched,bought
0,"6,7,8",NaN
1,"13,14,15",NaN
2,"22,23",NaN
3,"28,29,30,31,32,33",NaN
4,"40,41",NaN


In [5]:
from collections import Counter, OrderedDict

In [6]:
train_data.watched.values

array(['0,1,2,3,4,5', '9,10,11,9,11,12,9,11', '16,17,18,19,20,21', ...,
       '980,20025,980,20025,980,20025,980,20025',
       '8844,42500,8838,8172,29237,352,8847,6681,8827,8844',
       '39047,102806,27774'], dtype=object)

In [10]:
w_ids = [list(map(int, x.split(','))) for x in train_data.watched.values]
b_ids = [list(map(int, x.split(','))) for x in train_data.bought[~train_data.bought.isna()].values]

In [12]:
w_count = Counter()
for x in w_ids:
    w_count.update(x)

In [13]:
b_count = Counter()
for x in b_ids:
    b_count.update(x)

In [33]:
train = train_data.dropna()
test = test_data.dropna()

In [40]:
train = train.applymap(lambda x: list(map(int, x.split(','))))
test = test.applymap(lambda x: list(map(int, x.split(','))))

In [44]:
def sort_by_freq(lst, counts):
    return sorted(list(OrderedDict.fromkeys(lst)), key=lambda x: counts[x], reverse=True)

In [49]:
train['watched_sorted'] = train.watched.apply(lambda x: sort_by_freq(x, w_count))
train['watched_sorted_by_bought'] = train.watched.apply(lambda x: sort_by_freq(x, b_count))

In [58]:
train_by_watched_inter5 = train.loc[:, ['watched_sorted', 'bought']].apply(lambda x: set(x.watched_sorted[:5]) & set(x.bought), axis=1)
train_by_watched_inter1 = train.loc[:, ['watched_sorted', 'bought']].apply(lambda x: set(x.watched_sorted[:1]) & set(x.bought), axis=1)

train_by_bought_inter5 = train.loc[:, ['watched_sorted_by_bought', 'bought']].apply(lambda x: set(x.watched_sorted_by_bought[:5]) & set(x.bought), axis=1)
train_by_bought_inter1 = train.loc[:, ['watched_sorted_by_bought', 'bought']].apply(lambda x: set(x.watched_sorted_by_bought[:1]) & set(x.bought), axis=1)

In [56]:
train

,watched,bought,watched_sorted,watched_sorted_by_bought
7,"[59, 60, 61, 62, 60, 63, 64, 65, 66, 61, 67, 6...","[67, 60, 63]","[63, 64, 60, 61, 65, 66, 67, 68, 59, 62]","[60, 63, 67, 59, 61, 62, 64, 65, 66, 68]"
10,"[84, 85, 86, 87, 88, 89, 84, 90, 91, 92, 93, 86]",[86],"[85, 93, 89, 90, 84, 92, 86, 87, 91, 88]","[86, 85, 93, 84, 87, 88, 89, 90, 91, 92]"
19,"[138, 198, 199, 127]",[199],"[127, 138, 198, 199]","[138, 199, 127, 198]"
30,"[303, 304, 305, 306, 307, 308, 309, 310, 311, ...",[303],"[303, 306, 304, 307, 309, 310, 305, 308, 311, ...","[303, 304, 305, 306, 307, 308, 309, 310, 311, ..."
33,"[352, 353, 352]",[352],"[352, 353]","[352, 353]"
...,...,...,...,...
49943,"[41795, 4337, 4335, 4337, 4335, 24087, 4335, 4...",[4335],"[4335, 4337, 24073, 41795, 57885, 24087, 4344,...","[4335, 41795, 4337, 24087, 24073, 4344, 57885,..."
49964,"[6366, 15269, 6366, 5895, 6366, 5895]",[5895],"[5895, 6366, 15269]","[5895, 6366, 15269]"
49981,"[64552, 25931, 2807]","[25935, 2807]","[2807, 25931, 64552]","[2807, 64552, 25931]"
49991,"[91921, 20251, 5063, 21742, 5063, 20251, 34927]",[91921],"[5063, 21742, 20251, 91921, 34927]","[91921, 5063, 20251, 21742, 34927]"


In [64]:
def avrecall(intersected, bought):
    return (intersected.apply(len) / bought.apply(len)).mean()

In [66]:
def avprecision(intersected, n):
    return (intersected.apply(len) / n).mean() 

In [67]:
print('Train results sorted by watched:')
print(avrecall(train_by_watched_inter1, train.bought))
print(avprecision(train_by_watched_inter1, 1))
print(avrecall(train_by_watched_inter5, train.bought))
print(avprecision(train_by_watched_inter5, 5))

Train results sorted by watched:
0.4426343165949593
0.5121951219512195
0.8246918247126122
0.21252771618625918


In [76]:
print('Train results sorted by bought:')
print(avrecall(train_by_bought_inter1, train.bought))
print(avprecision(train_by_bought_inter1, 1))
print(avrecall(train_by_bought_inter5, train.bought))
print(avprecision(train_by_bought_inter5, 5))

Train results sorted by bought:
0.6884494924267653
0.8037694013303769
0.9263073024228787
0.2525498891352649


In [77]:
test

,watched,bought
7,"[63, 68, 69, 70, 66, 61, 59, 61, 66, 68]","[66, 63]"
14,"[158, 159, 160, 159, 161, 162]",[162]
19,"[200, 201, 202, 203, 204]","[201, 205]"
34,"[371, 372, 371]","[371, 373]"
40,[422],[422]
...,...,...
49943,"[2859, 2854, 88887, 96997, 4439, 28645, 99975,...","[24907, 102691, 18496, 4333]"
49944,"[77655, 23249, 1306, 47450, 26157, 58205, 4745...","[58205, 3111, 69482]"
49945,"[60538, 44430, 66252, 44430, 60538, 66251]","[66252, 44430]"
49946,"[49815, 76363]",[49815]


In [80]:
test['watched_sorted'] = test.watched.apply(lambda x: sort_by_freq(x, w_count))
test['watched_sorted_by_bought'] = test.watched.apply(lambda x: sort_by_freq(x, b_count))

In [81]:
test

,watched,bought,watched_sorted,watched_sorted_by_bought
7,"[63, 68, 69, 70, 66, 61, 59, 61, 66, 68]","[66, 63]","[63, 68, 66, 61, 59, 69, 70]","[63, 68, 69, 70, 66, 61, 59]"
14,"[158, 159, 160, 159, 161, 162]",[162],"[158, 162, 160, 159, 161]","[158, 162, 160, 159, 161]"
19,"[200, 201, 202, 203, 204]","[201, 205]","[204, 202, 203, 200, 201]","[204, 202, 200, 201, 203]"
34,"[371, 372, 371]","[371, 373]","[371, 372]","[371, 372]"
40,[422],[422],[422],[422]
...,...,...,...,...
49943,"[2859, 2854, 88887, 96997, 4439, 28645, 99975,...","[24907, 102691, 18496, 4333]","[4335, 4333, 24907, 606, 2854, 4439, 96997, 34...","[4335, 4333, 606, 2859, 2854, 88887, 96997, 44..."
49944,"[77655, 23249, 1306, 47450, 26157, 58205, 4745...","[58205, 3111, 69482]","[1262, 16237, 7318, 1261, 1306, 3111, 26157, 4...","[1262, 1306, 13520, 77655, 23249, 47450, 26157..."
49945,"[60538, 44430, 66252, 44430, 60538, 66251]","[66252, 44430]","[60538, 44430, 66252, 66251]","[60538, 44430, 66252, 66251]"
49946,"[49815, 76363]",[49815],"[49815, 76363]","[49815, 76363]"


In [82]:
test_by_watched_inter5 = test.loc[:, ['watched_sorted', 'bought']].apply(lambda x: set(x.watched_sorted[:5]) & set(x.bought), axis=1)
test_by_watched_inter1 = test.loc[:, ['watched_sorted', 'bought']].apply(lambda x: set(x.watched_sorted[:1]) & set(x.bought), axis=1)

test_by_bought_inter5 = test.loc[:, ['watched_sorted_by_bought', 'bought']].apply(lambda x: set(x.watched_sorted_by_bought[:5]) & set(x.bought), axis=1)
test_by_bought_inter1 = test.loc[:, ['watched_sorted_by_bought', 'bought']].apply(lambda x: set(x.watched_sorted_by_bought[:1]) & set(x.bought), axis=1)

In [83]:
print('Test results sorted by watched:')
print(avrecall(test_by_watched_inter1, test.bought))
print(avprecision(test_by_watched_inter1, 1))
print(avrecall(test_by_watched_inter5, test.bought))
print(avprecision(test_by_watched_inter5, 5))

Test results sorted by watched:
0.41733266203252534
0.48130968622100956
0.8000340663538579
0.2037653478854079


In [84]:
print('Test results sorted by bought:')
print(avrecall(test_by_bought_inter1, test.bought))
print(avprecision(test_by_bought_inter1, 1))
print(avrecall(test_by_bought_inter5, test.bought))
print(avprecision(test_by_bought_inter5, 5))

Test results sorted by bought:
0.4606201666660294
0.5276944065484311
0.8201874337490194
0.21009549795362173
